In [1]:
import cv2
import os
import numpy as np
import glob
import random
import time 
import matplotlib.pyplot as plt
import re
from PIL import Image
import re

In [2]:
video_dir='/content/drive/MyDrive/Sproj2/videos/Gulberg_night.mp4'
frames_dir='/content/frames_night_gulberg/'
resized_frames_dir='/content/resized_night_gulberg/'
results_dir='/content/results_night_gulberg/'

## **Generate frames from video**

In [27]:
!rm -rf '/content/resized_night_gulberg'

In [3]:
# Generate frames from video
cap= cv2.VideoCapture(video_dir)
i=0
!mkdir '/content/frames_night_gulberg/'
count=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    # value of i=1800 to save 1800/5 frames (360 frames)
    # if i==1800:
    #   break
    if count==360:
      break
    if i>=2700:
      if i%5==0:
        cv2.imwrite(frames_dir+str(count)+'.jpg',frame)
        count+=1
    i+=1

 
cap.release()
cv2.destroyAllWindows()


# **Run the cells below to resize images**

In [4]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key)] 
    return sorted(data, key=alphanum_key)

In [ ]:
# Resize all the frames to size of (1024,1920,3) from (1080,1920,3)
# Pythono3 code to rename multiple 
# files in a directory or folder 

# Function to rename multiple files 
!mkdir '/content/resized_night_gulberg/'
def main(): 

	for count, filename in enumerate(sorted_alphanumeric(os.listdir(frames_dir))):
		image = Image.open(frames_dir+filename)
		new_image = image.resize((1024, 1920))
		new_image.save(resized_frames_dir+str(count)+'.jpg')
		print(image.size) # Output: (1200, 776)
		print(new_image.size) # Output: (400, 400)
		
		# rename() function will 
		# rename all the files 
		 

# Driver Code 
if __name__ == '__main__': 
	
	# Calling main() function 
	main() 


# **Yolo Detection**

In [7]:
!cp -r '/content/drive/MyDrive/Sproj2/vehicle-yolov3_9500.weights' '/content/'

In [6]:
%%writefile /content/vehicle-yolov3.cfg
[net]
# Testing
# batch=64
subdivisions=8
# Training
batch=24
subdivisions=2
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=16
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=1

[convolutional]
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky

###########

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=36
activation=linear



[yolo]
mask = 3,4,5
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes=7
num=6
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1

[route]
layers = -4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 8

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=36
activation=linear

[yolo]
mask = 0,1,2
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes=7
num=6
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1

Writing /content/vehicle-yolov3.cfg


In [8]:
# Load Yolo
load_dir='/content/'
yolo_net = cv2.dnn.readNet( load_dir + "vehicle-yolov3_9500.weights", load_dir + "vehicle-yolov3.cfg")

layer_names = yolo_net.getLayerNames()

output_layers = [layer_names[i[0] - 1] for i in yolo_net.getUnconnectedOutLayers()]

In [9]:
# Object Name
classes = ["Bike","Bus","Car","MiniBus","MiniTruck","Rickshaw","Truck"]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [29]:
# This will create a text file containing cordinates of bounding boxes of vehicles to be used for counting.
def Object_Detection(img,count,f):
    
    # for key in list(current_frame.keys()):
        
    begin = time.time() 
    
    # Image 
    # img = current_frame[key]
    height, width, channels = img.shape
    # print(height,width,channels)
    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    yolo_net.setInput(blob)
    outs = yolo_net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = [] 
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.8:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                # for i in range(15):
                #   rand_point_x=random.randint(x,x+w)
                #   rand_point_y=random.randint(y,y+h)
                #   f.write(str(rand_point_x)+' '+str(rand_point_y) + ' ')
                label=classes[class_id]
                f.write('{} {} {} {} {} '.format(x,y,w,h,label))

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    f.write('\n')
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.ACCESS_MASK
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            # cv2.putText(img,'x',(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,color = (255, 0, 0))
            # cv2.putText(img,'x',(x+w,y+h),cv2.FONT_HERSHEY_SIMPLEX,1,color = (255, 0, 0))
            cv2.rectangle(img, (x, y), (x + w, y + h), (0,0,255), 1)
            cv2.putText(img, label+ str(np.around(confidences[i],2)*100) + "%", (x, y-5), font, 0.4, (0,0,255), 1)

    # cv2.imshow(key, img)
    # print(key)
    cv2.imwrite(results_dir+str(count)+'.jpg',img)
    cv2.waitKey(2)

In [30]:
import time
t0=time.time()
count=0
f=open('cords_gulberg_night.txt','w')
!mkdir '/content/results_night_gulberg'
for i,item in enumerate(sorted_alphanumeric(os.listdir(resized_frames_dir))):
  img=cv2.imread(resized_frames_dir+item)
  Object_Detection(img,count,f)
  count+=1
f.close()
t1=time.time()
total_time=t1-t0
print(total_time)

77.55743622779846


In [ ]:
!zip -r 'frames_night_gulberg.zip' '/content/resized_night_gulberg'

# Vehicle Counting

In [ ]:
import numpy as np
import os
import random
import cv2
import re

The function below reads the cordinates of the ends of the two lines. 
Line 1 of txt file contains points of bottom line
Line 2 of txt file contains points of top line

In [ ]:
def read_line_cords(f):
    line1_cords=f.readline().split()
    line2_cords=f.readline().split()
    return line1_cords,line2_cords

In [ ]:
# This function checks if the point lies within the two lines. Returns 1 if true, else returns 0
def check_insider(cords,line_cords):
#    y=0.074x + 412.595 line1
    # y=0.074x + 399.684 line 2
    x,y=cords
    y1,y2=line_cords
    y1=int(y1)
    y2=int(y2)
    # print(y1,y2)
    # print(y)
     
    # y1= 0.079*x + 399.684
    if y<y1 and y>y2:
        return 1
    else:
        return 0

In [ ]:
# make_flow function is used to read flo file and returns numpy array
def make_flow(flo):
    tag = np.fromfile(flo, np.float32, count=1)[0]
    width = np.fromfile(flo, np.int32, count=1)[0]
    # print(width)
    height = np.fromfile(flo, np.int32, count=1)[0]
    # print(height)
    nbands = 2
    tmp = np.fromfile(flo, np.float32, count= nbands * width * height)
    flow = np.resize(tmp, (int(height), int(width), int(nbands)))
    return flow

In [ ]:

# Read flowframe
# From flownet code
# Read the flowframe and return numpy array


 


def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)


# Commented flow part
# flo_names=sorted_alphanumeric(os.listdir('/content/drive/MyDrive/Sproj2/Flo_600'))
# print(len(flo_names))
# flo_np_arr=[]
# for i in range(len(flo_names)-300):
#     x = open(f"/content/drive/MyDrive/Sproj2/Flo_600/{flo_names[i]}", "r") 
#     flowf = make_flow(x)
#     print('flow',flowf.shape)
#     flo_np_arr.append(flowf)
    # print(flowf.shape)


# print(flowf[8][323])
# print(flo_np_arr)

def count_classes(label,count_c):
  if label=='Bike':
    count_c[0]+=1
  elif label=='Car':
    count_c[1]+=1
  elif label=='Minibus':
    count_c[2]+=1
  elif label=='MiniTruck':
    count_c[3]+=1
  elif label=='Truck':
    count_c[4]+=1
  elif label=='Rickshaw':
    count_c[5]+=1
  elif label=='Bus':
    count_c[6]+=1
  return count_c

def read_coords(filename):
    count_c=[0,0,0,0,0,0,0]
    f=open(filename,'r')
    line_cords_filename='./day_gulberg.txt'
    f2=open(line_cords_filename,'r')
    line1_cords,line2_cords=read_line_cords(f2)
    images_coords=f.read().split('\n')
    car_cords=[]
    cars_dict={}
    all_image_vel_x=[]
    all_image_vel_y=[]
    image_names=sorted_alphanumeric(os.listdir('D:/sproj 2/Code/Dataset/Day_gulberg/frames_gulberg_day/content/resized_gulberg_day'))
    count=0
    for i in range(len(images_coords)-1):
        img=cv2.imread('D:/sproj 2/Code/Dataset/Day_gulberg/frames_gulberg_day/content/resized_gulberg_day/'+image_names[i])
        print('image',img.shape)
        points_arr=images_coords[i].split()
        # print(len(points_arr))
        all_car_vel_x=[]
        all_car_vel_y=[]
        for j in range(1,len(points_arr)+1):
            if j%5!=0:
                car_cords.append(points_arr[j-1])
            else:
                car_cords.append(points_arr[j-1])
                sin_car_vel_x=[]
                sin_car_vel_y=[]
                check=[]
                num=0
                for x in range(50):
                    # rand_point_x=1930
                    # while rand_point_x > 1920:
                    if (int(car_cords[0])+int(car_cords[2]) > 1920) or (int(car_cords[1])+int(car_cords[3])>1024):
                        # print("x+w ... ",int(car_cords[0])+int(car_cords[2]))
                        # print("y+h ... ",int(car_cords[1]) + int(car_cords[3]))
                        # print('x=',x)
                        # print(num)
                        num+=1
                        continue
                    else:
                        # print('here')
                        # print('x',car_cords[1],int(car_cords[1]) + int(car_cords[3]))
                        # print('y',car_cords[0],int(car_cords[0])+int(car_cords[2]))
                        rand_point_x=random.randint(round(int(car_cords[1])+((0.40)*int(car_cords[3]))),round(int(car_cords[1])+((0.60)*int(car_cords[3]))))
                        rand_point_y=random.randint(round(int(car_cords[0])+((0.20)*int(car_cords[2]))),round(int(car_cords[0])+((0.80)*int(car_cords[2]))))
                        check.append(check_insider((rand_point_y,rand_point_x),(line1_cords[1],line2_cords[1])))
                        if check_insider((rand_point_y,rand_point_x),(line1_cords[1],line2_cords[1])):
                            img=cv2.circle(img,(rand_point_y,rand_point_x),4,(255,0,0),3)

                        # commented flow part
                        # vel_x=flo_np_arr[i][rand_point_x][rand_point_y][1]
                        # vel_y=flo_np_arr[i][rand_point_x][rand_point_y][0]
                        # # print(vel_y)
                        # sin_car_vel_x.append(vel_x)
                        # sin_car_vel_y.append(vel_y)
                

                sum_check=sum(check)/50
                if sum_check>0.50:
                    count+=1
                    count_c=count_classes(car_cords[4],count_c)


                # all_car_vel_x.append(np.mean(sin_car_vel_x))
                # all_car_vel_y.append(np.mean(sin_car_vel_y))

                font = cv2.FONT_HERSHEY_SIMPLEX
                fontScale = 1
                # img=cv2.putText(img,'x: '+str(np.mean(sin_car_vel_x)),(round(int(car_cords[0])+(0.25)*int(car_cords[2])),round(int(car_cords[1])+(0.25)*int(car_cords[3])+35)),font,fontScale,color = (255, 0, 0))
                # img=cv2.putText(img,'y: '+str(np.mean(sin_car_vel_y)),(round(int(car_cords[0])+(0.25)*int(car_cords[2])),round(int(car_cords[1])+(0.25)*int(car_cords[3]))),font,fontScale,color = (255, 0, 0))
                img=cv2.putText(img,car_cords[4],(round(int(car_cords[0])+(0.25)*int(car_cords[2])),round(int(car_cords[1])+(0.25)*int(car_cords[3])+35)),font,fontScale,color = (255, 0, 0))
                img=cv2.rectangle(img,((round(int(car_cords[0])+(0.20)*int(car_cords[2])),round(int(car_cords[1])+(0.40)*int(car_cords[3])))),(round(int(car_cords[0])+(0.80)*int(car_cords[2])),round(int(car_cords[1])+(0.60)*int(car_cords[3]))),(255, 0, 0),1)
                img=cv2.rectangle(img,(int(car_cords[0]),int(car_cords[1])),(int(car_cords[0])+int(car_cords[2]),int(car_cords[1])+int(car_cords[3])),(255, 0, 0),1)
                # print(all_car_vel)
                car_cords=[]
        # all_image_vel_x.append(all_car_vel_x)
        # all_image_vel_y.append(all_car_vel_y)
        color=(172,172,172)
        color=(255,0,0)
        image=cv2.line(img,(int(line1_cords[0]),int(line1_cords[1])),(int(line1_cords[2]),int(line1_cords[3])),(0,0,255),5)
        image=cv2.line(img,(int(line2_cords[0]),int(line2_cords[1])),(int(line2_cords[2]),int(line2_cords[3])),(0,0,255),5)
        image=cv2.putText(image,f'Car: {count_c[1]}',(30,30),font,fontScale,color = color,thickness=2)
        image=cv2.putText(image,f'Bike: {count_c[0]}',(250,30),font,fontScale,color = color,thickness=2)
        image=cv2.putText(image,f'Rickshaw: {count_c[5]}',(470,30),font,fontScale,color = color,thickness=2)
        image=cv2.putText(image,f'Bus: {count_c[6]}',(690,30),font,fontScale,color = color,thickness=2)
        image=cv2.putText(image,f'Mini Bus: {count_c[2]}',(910,30),font,fontScale,color = color,thickness=2)
        image=cv2.putText(image,f'Mini Truck: {count_c[3]}',(1130,30),font,fontScale,color = color,thickness=2)
        image=cv2.putText(image,f'Truck: {count_c[4]}',(1350,30),font,fontScale,color = color,thickness=2)
        
        cv2.imwrite('./results_day_gulberg/'+str(i)+'.jpg',image)      
    #     print("cars",len(cars))
    #     images.append(cars)
    #     cars = []
    # print("images:",len(images))
    # return all_image_vel_x,all_image_vel_y,count
    return count
# velx,vely,count=read_coords('/content/cords_600.txt')
count=read_coords('D:/sproj 2/Code/cords_gulberg_day.txt')


# print('x: ',velx)
# print('y: ',vely)
print(count)

